In [1]:
import pandas as pd
import numpy as np

In [ ]:
## 负载曲线数据格式
## 0 - 所
## 1 - 用户类型
## 2 - 用户编号
## 3 - 用户名称
## 4 - 表计资产号
## 5 - 数据日期
## 6 - CT
## 7 - PT
## 8 - A相电压
## 9 - B相电压
## 10 - C相电压
## 11 - A相电流
## 12 - B相电流
## 13 - C相电流
## 14 - 零序电流
## 15 - 有功功率
## 16 - A相有功功率
## 17 - B相有功功率
## 18 - C相有功功率
## 19 - 无功功率
## 20 - A相无功功率
## 21 - B相无功功率
## 22 - C相无功功率

In [ ]:
## SG186 数据格式表
## 0 - 用户编号
## 1 - 线路名称
## 2 - 资产编号
## 3 - 出厂编号
## 4 - 综合倍率
## 5 - 变压器容量
## 6 - 接线方式
## 7 - 电压
## 8 - 标定电流

In [ ]:
## Constants

## 错误编码
ERROR_SUCCEED = 0
ERROR_CURRENT_MANUAL_CHECK = 1
ERROR_CURRENT_SAME_PARTITION = 2

## A, B, C 相对索引
PHASE_A_INDEX = 0
PHASE_B_INDEX = 1
PHASE_C_INDEX = 2

## 接线模式
LINE_MODE_3P4L = 0 # 三相四线
LINE_MODE_3P3L = 1 # 三相三线

## 相序
PHASE_SEQ_POSITIVE = 0 # 正相序
PHASE_SEQ_NEGATIVE = 1 # 逆相序

## 电压滞后
VOLTAGE_PHASE_DELAY = [
    [[0.0, 120.0, 240.0], [0.0, 240.0, 120.0]],
    [[330.0, 270.0], [30.0, 60.0]],
]

## 顺时针电流分布
CURRENT_CLOCKWISE = ["+Ia", "-Ic", "+Ib", "-Ia", "+Ic", "-Ib"]
CURRENT_PARTITION_MAX = 6

## 正确接线矩阵
CORRECT_LINE_MATRIX = [
    [[0, 2, 4], [0, 4, 2]], # 三相四线
    [[0, 4], [0, 2]]  # 三相三线
]

In [ ]:
## Variables

## 单相电流
currentA = 0.0
currentB = 0.0
currentC = 0.0

## 单相电压
voltageA = 0.0
voltageB = 0.0
voltageC = 0.0

## 有功功率
activePowerA = 0.0
activePowerB = 0.0
activePowerC = 0.0
activePowerTotal = 0.0

## 无功功率
reactivePowerA = 0.0
reactivePowerB = 0.0
reactivePowerC = 0.0
reactivePowerTotal = 0.0

## 变压器参数
transformerDeviceID = None # 变压器设备编号
transformerLineMode = None # 接线模式
transformerCapacity = 0.0 # 变压器容量
ratedCurrent = 1.5 # 单位：A，默认值 1.5
ratedVoltage = 220.0 # 单位：V，默认值 220

In [ ]:
## 控制参数
currentLoadRatio = 0.3
phaseSeqRation = 0.1
currentADelayMinAngle = 8.0 # A 相电流滞后 A 相电压的最小角度

In [ ]:
def printPhaseSequence(seq):
    if seq == PHASE_SEQ_POSITIVE:
        return "正相序"
    elif seq == PHASE_SEQ_NEGATIVE:
        return "逆相序"
    else:
        return "未知相序"
    
def printLineMode(mode):
    if mode == LINE_MODE_3P4L:
        print("接线模式：\t\t\t三相四线")
    elif mode == LINE_MODE_3P3L:
        print("接线模式：\t\t\t三相三线")
    else:
        print("接线模式：\t\t\t未知")

def printVariables():
    print("======== 变压器参数 ========")
    print("变压器设备编号：\t\t\t", transformerDeviceID)
    printLineMode(transformerLineMode)
    print("变压器容量：\t\t\t", transformerCapacity)
    
    print("标定电流：\t\t\t", ratedCurrent)
    print("标定电压：\t\t\t", ratedVoltage)
    print()
    
    print("======== 电表读数 ========")
    print("A相电压：\t\t\t", voltageA)
    print("B相电压：\t\t\t", voltageB)
    print("C相电压：\t\t\t", voltageC)
    print()
    
    print("A相电流：\t\t\t", currentA)
    print("B相电流：\t\t\t", currentB)
    print("C相电流：\t\t\t", currentC)
    print()
    
    print("A相有功功率：\t\t\t", activePowerA)
    print("B相有功功率：\t\t\t", activePowerB)
    print("C相有功功率：\t\t\t", activePowerC)
    print("有功功率：\t\t\t", activePowerTotal)
    print()
    
    print("A相无功功率：\t\t\t", reactivePowerA)
    print("B相无功功率：\t\t\t", reactivePowerB)
    print("C相无功功率：\t\t\t", reactivePowerC)
    print("无功功率：\t\t\t", reactivePowerTotal)
    print()

In [ ]:
## 根据 表计资产号 从 SG186 系统中获取基础信息
def informationInSG186System(deviceID):
    return sg186DF.where(sg186DF['资产编号'] == deviceID).dropna()

In [ ]:
import re

## 解析 1.5(6)A 格式的标定电流
## 1 - Calibrated Current (1.5)
## 2 - Rated Current (6)
## 使用示例 
##     calibratedCurrent, ratedCurrent = parseCalibratedRatedCurrent("1.5(6)A")
def parseCalibratedRatedCurrent(label):
    try:
        result = re.match('^(\d+[\.]*[\d]*)\((\d+)\)A$', label)
        return result.group(1), result.group(2)
    except AttributeError:
        return 0, 0

## 解析 3x380/220 格式的三相四线电压值
## 1 - 火火电压 (380)
## 2 - 火零电压 (220)
def parseVoltage3P4L(label):
    try:
        result = re.match('^3x(\d+)/(\d+)$', label)
        return result.group(1), result.group(2)
    except AttributeError:
        return 0, 0

## 解析 3x100V 格式的三相三线电压值
## 1 - 火火电压 (100)
def parseVoltage3P3L(label):
    try:
        result = re.match('^3x(\d+)V$', label)
        return result.group(1)
    except AttributeError:
        return 0, 0

In [ ]:
## 电流滞后电压角度计算
def voltageCurrentPhaseDistance(active, reactive):
    if (active == 0) and (reactive == 0):
        return None

    degree = np.degrees(np.arccos(np.abs(active) / np.sqrt(active ** 2 + reactive ** 2)))
    
    if active >= 0:
        if reactive >= 0:
            return degree
        else:
            return 360.0 - degree
    else:
        if reactive >= 0:
            return 180.0 - degree
        else:
            return 180.0 + degree

In [ ]:
## 加载模拟数据
def initialVariables(data):
    global transformerDeviceID, transformerLineMode, transformerCapacity, ratedCurrent, ratedVoltage
    global currentA, currentB, currentC, voltageA, voltageB, voltageC
    global activePowerA, activePowerB, activePowerC, activePowerTotal
    global reactivePowerA, reactivePowerB, reactivePowerC, reactivePowerTotal
    
    ## 变压器参数
    transformerDeviceID = data[4]
    
    # 获取 SG186 信息
    sg186Info = informationInSG186System(transformerDeviceID)
    
    if sg186Info.iloc[0][6] == "三相三线":
        transformerLineMode = LINE_MODE_3P3L
    elif sg186Info.iloc[0][6] == "三相四线":
        transformerLineMode = LINE_MODE_3P4L
    else:
        transformerLineMode = None

    transformerCapacity = float(sg186Info.iloc[0][5])
    ratedCurrent, _ = parseCalibratedRatedCurrent(sg186Info.iloc[0][8])
    ratedCurrent = float(ratedCurrent)

    if transformerLineMode == LINE_MODE_3P3L:
        ratedVoltage = parseVoltage3P3L(sg186Info.iloc[0][7])
    elif transformerLineMode == LINE_MODE_3P4L:
        _, ratedVoltage = parseVoltage3P4L(sg186Info.iloc[0][7])
    else:
        ratedVoltage = 0.0
    ratedVoltage = float(ratedVoltage)
    
    currentA = data[11]
    currentB = data[12]
    currentC = data[13]

    ## 单相电压
    voltageA = data[8]
    voltageB = data[9]
    voltageC = data[10]

    ## 有功功率
    activePowerA = data[16]
    activePowerB = data[17]
    activePowerC = data[18]
    activePowerTotal = data[15]

    ## 无功功率
    reactivePowerA = data[20]
    reactivePowerB = data[21]
    reactivePowerC = data[22]
    reactivePowerTotal = data[19]

In [ ]:
## Filters

## 单相电流为 0 或 大于额定电流的指定比率，默认比率为 30%
def isPhaseLoadFine(phaseCurrent, current, rate = 0.3):
    limit = current * rate
    
    return (phaseCurrent == 0) or (phaseCurrent > limit)

## 相序异常检测
## A - A 相功率
## B - B 相功率
## C - C 相功率
## T - 总相功率
## rate - 比率值，默认 10%
def isPhaseSequenceFine(A, B, C, T, rate = 0.1):
    return np.abs((A + B + C) - T) > T * rate

In [ ]:
## 计算电流相对于 A 相电压的角度

def currentPhaseBaseA3P4L(A, B, C, Seq):
    maxDegree = 360.0
    delayMap = VOLTAGE_PHASE_DELAY[LINE_MODE_3P4L][Seq]

    return [
        (A + delayMap[PHASE_A_INDEX]) % maxDegree,
        (B + delayMap[PHASE_B_INDEX]) % maxDegree,
        (C + delayMap[PHASE_C_INDEX]) % maxDegree
    ]

def currentPhaseBaseA3P3L(A, B, C, Seq):
    maxDegree = 360.0
    delayMap = VOLTAGE_PHASE_DELAY[LINE_MODE_3P3L][Seq]

    return [
        (A + delayMap[PHASE_A_INDEX]) % maxDegree,
        (B + delayMap[PHASE_B_INDEX]) % maxDegree,
        (C + delayMap[PHASE_C_INDEX]) % maxDegree
    ]

def currentPhaseBaseA(A, B, C):
    delayMap = None
    result = [None, None]

    if transformerLineMode == LINE_MODE_3P4L:
        result[PHASE_SEQ_POSITIVE] = currentPhaseBaseA3P4L(A, B, C, PHASE_SEQ_POSITIVE)
        result[PHASE_SEQ_NEGATIVE] = currentPhaseBaseA3P4L(A, B, C, PHASE_SEQ_NEGATIVE)
    elif transformerLineMode == LINE_MODE_3P3L:
        result[PHASE_SEQ_POSITIVE] = currentPhaseBaseA3P3L(A, B, C, PHASE_SEQ_POSITIVE)
        result[PHASE_SEQ_NEGATIVE] = currentPhaseBaseA3P3L(A, B, C, PHASE_SEQ_NEGATIVE)
    else:
        pass
    
    return result

In [ ]:
## 获取各相电流所在分区号
def currentPartition(current, currentAMinAngle):
    if current >= currentAMinAngle:
        return int((current - currentAMinAngle) / 60.0)
    else:
        return int((current + 360.0 - currentAMinAngle) / 60.0)

## 获取全部电流分区信息
## currents -  [A, B, C]
## currentAMinAngle - A 相电流滞后 A 相电压的最小角度
def currentPartitionsAnalysis(currents, currentAMinAngle):
    partitions = [None, None, None]
    
    partitions[PHASE_A_INDEX] = currentPartition(currents[PHASE_A_INDEX], currentAMinAngle)
    partitions[PHASE_B_INDEX] = currentPartition(currents[PHASE_B_INDEX], currentAMinAngle)
    partitions[PHASE_C_INDEX] = currentPartition(currents[PHASE_C_INDEX], currentAMinAngle)
    
    return partitions

In [ ]:
## 电流检测
def currentVerify(phases, partitions):
    l = len(phases)
    
    for i in range(l - 1):
        for j in range(i + 1, l):
            diff = np.abs(phases[i] - phases[j])
            if diff <= 5:
                return ERROR_CURRENT_MANUAL_CHECK
            
            if (diff >= 175) and (diff < 185):
                return ERROR_CURRENT_MANUAL_CHECK
    
    for i in range(l - 1):
        for j in range(i + 1, l):
            if partitions[i] == partitions[j]:
                return ERROR_CURRENT_SAME_PARTITION
    
    return ERROR_SUCCEED

def currentLineVerify(partitions, lineMode, seq):
    return partitions == CORRECT_LINE_MATRIX[lineMode][seq]

In [ ]:
### Main Procedure
def process():
    # Filter 1
    if isPhaseLoadFine(currentA, ratedCurrent, currentLoadRatio) == False:
        print("A相电流负荷不正常")
        return
    
    if isPhaseLoadFine(currentB, ratedCurrent, currentLoadRatio) == False:
        print("B相电流负荷不正常")
        return

    if isPhaseLoadFine(currentC, ratedCurrent, currentLoadRatio) == False:
        print("C相电流负荷不正常")
        return
    
    # Filter 2
    if isPhaseSequenceFine(activePowerA, activePowerB, activePowerC, activePowerTotal) == False:
        print("相序异常")
        return
    
    currentVoltageDelayA = voltageCurrentPhaseDistance(activePowerA, reactivePowerA)
    currentVoltageDelayB = voltageCurrentPhaseDistance(activePowerB, reactivePowerB)
    currentVoltageDelayC = voltageCurrentPhaseDistance(activePowerC, reactivePowerC)
    print("电流滞后电压角度[A, B, C]:", [currentVoltageDelayA, currentVoltageDelayB, currentVoltageDelayC])
    currentPhases = currentPhaseBaseA(currentVoltageDelayA, currentVoltageDelayB, currentVoltageDelayC)
    print("各相电流滞后 A 相电压角度 - 正相序 - [A, B, C]:", currentPhases[PHASE_SEQ_POSITIVE])
    print("各相电流滞后 A 相电压角度 - 逆相序 - [A, B, C]:", currentPhases[PHASE_SEQ_NEGATIVE])
    
    currentPartitions = [None, None]
    currentPartitions[PHASE_SEQ_POSITIVE] = currentPartitionsAnalysis(currentPhases[PHASE_SEQ_POSITIVE], currentADelayMinAngle)
    currentPartitions[PHASE_SEQ_NEGATIVE] = currentPartitionsAnalysis(currentPhases[PHASE_SEQ_NEGATIVE], currentADelayMinAngle)
    
    print("各相电流所在分区 - 正相序 - [A, B, C]:", currentPartitions[PHASE_SEQ_POSITIVE])
    print("各相电流所在分区 - 逆相序 - [A, B, C]:", currentPartitions[PHASE_SEQ_NEGATIVE])
    
    for seq in [PHASE_SEQ_POSITIVE, PHASE_SEQ_NEGATIVE]:
        print()
        print(printPhaseSequence(seq), " - 检测开始")
        phases = currentPhases[seq]
        partitions = currentPartitions[seq]
        
        print("电流角：", phases)
        print("分区：", partitions)
        
        result = currentVerify(phases, partitions)
        if result == ERROR_CURRENT_MANUAL_CHECK:
            print("一相电流二次线串接两个元件以及存在两根火线用电设备，请人工检查分析")
            continue
        elif result == ERROR_CURRENT_SAME_PARTITION:
            print("三相负荷严重不平衡，请人工检查分析")
            continue
        else:
            if currentLineVerify(partitions, transformerLineMode, seq) == True:
                print("检查结果：接线正确")
            else:
                print("检查结果：错误接线")

In [ ]:
### Execution
initialVariables(loadDF.loc[0])

In [ ]:
printVariables()

In [ ]:
process()